In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install necessary libraries
!pip install flask-ngrok
!pip install tensorflow
!pip install numpy
!pip install pickle5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255318 sha256=cfd8ba88957c2980ee1351ef6c59399effb1fa5025a6b43ec11c9ff41e0e96f8
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5


In [ ]:
!pip install pyngrok

In [ ]:
!ngrok authtoken 2eCNXiX4M2ibFVeKVGrIaF5mDFy_7VaCxWj59uQLxoW6ZxeDw

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)

# Define file paths in Google Drive
model_path = '/content/drive/My Drive/Emotion/emotion_detection_model.h5'
tokenizer_path = '/content/drive/My Drive/Emotion/tokenizer.pickle'
label_encoder_path = '/content/drive/My Drive/Emotion/label_encoder.pickle'

# Load the saved model
try:
    model = tf.keras.models.load_model(model_path)
    print("Model loaded successfully")
except Exception as e:
    print(f"Error loading model: {e}")
    exit(1)

# Load the tokenizer
try:
    with open(tokenizer_path, 'rb') as handle:
        tokenizer = pickle.load(handle)
    print("Tokenizer loaded successfully")
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    exit(1)

# Load the label encoder
try:
    with open(label_encoder_path, 'rb') as handle:
        label_encoder = pickle.load(handle)
    print("Label encoder loaded successfully")
except Exception as e:
    print(f"Error loading label encoder: {e}")
    exit(1)

# Tokenize and convert text to sequence
def tokenize_text(text):
    input_seq = tokenizer.texts_to_sequences([text])
    max_length = 100
    input_seq = pad_sequences(input_seq, maxlen=max_length, padding='post')
    print("Input Sequence Shape:", input_seq.shape)
    return input_seq

# Define emotion categories and their integer mappings
emotion_categories = {
    'anger': 0, 'boredom': 1, 'empty': 2, 'enthusiasm': 3, 'fun': 4,
    'happiness': 5, 'hate': 6, 'love': 7, 'neutral': 8, 'relief': 9,
    'sadness': 10, 'surprise': 11, 'worry': 12
}

# Predict emotion
def predict_emotion(input_text):
    try:
        print("Tokenizing text...")
        input_seq = tokenize_text(input_text)
        if input_seq is None:
            return "Error"

        print(input_text)
        print("Predicting probabilities...")
        predicted_probabilities = model.predict(input_seq)
        print("Predicted Probabilities Shape:", predicted_probabilities.shape)
        print("Predicted Probabilities:", predicted_probabilities)

        predicted_emotion_index = np.argmax(predicted_probabilities)
        print("Predicted Emotion Index:", predicted_emotion_index)

        predicted_emotion = list(emotion_categories.keys())[predicted_emotion_index]
        print("Predicted Emotion:", predicted_emotion)
        return emotion_categories[predicted_emotion]
    except Exception as e:
        print("Error during prediction:", e)
        return "Error"

# Define Flask route
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.get_json()
        print("Received JSON Data:", data)  # Print entire received JSON data

        input_text = data['input_text']
        print("Received Input:", input_text)  # Print input_text here

        predicted_emotion = predict_emotion(input_text)
        response_data = {'index': predicted_emotion}  # Prepare response data

        return jsonify(response_data)
    except KeyError:
        print("Error: 'input_text' not found in request JSON")
        return jsonify({'error': "'input_text' not found in request"}), 400
    except Exception as e:
        print("Error during request handling:", e)
        return jsonify({'error': 'An error occurred'}), 500

# Start ngrok tunnel
public_url = ngrok.connect(addr='5000', bind_tls=True)
print('Public URL:', public_url)

# Run Flask app
if __name__ == '__main__':
    app.run()


Model loaded successfully
Tokenizer loaded successfully
Label encoder loaded successfully
Public URL: NgrokTunnel: "https://67a7-35-234-42-225.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://67a7-35-234-42-225.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
Received JSON Data: {'input_text': 'i am'}
Received Input: i am
Tokenizing text...
Input Sequence Shape: (1, 100)
i am
Predicting probabilities...
1/1 [==============================] - 0s 316ms/step


INFO:werkzeug:127.0.0.1 - - [16/Apr/2024 04:36:24] "POST /predict HTTP/1.1" 200 -


Predicted Probabilities Shape: (1, 13)
Predicted Probabilities: [[1.04420685e-07 1.62605476e-12 2.01868591e-11 2.36528783e-11
  2.84722887e-06 2.12208548e-07 1.27821504e-08 6.47760179e-10
  9.99994874e-01 1.87244916e-06 3.13822568e-09 3.17274623e-10
  1.05100781e-12]]
Predicted Emotion Index: 8
Predicted Emotion: neutral
